In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_bootstrap import Bootstrap
from wtforms import StringField,PasswordField,SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Length, ValidationError, Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime
import pandas as pd
import sqlite3

app = Flask(__name__)

bootstrap = Bootstrap(app)
bcrypt = Bcrypt (app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
app.config['SECRET_KEY'] = 'thisissecret'

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'
    
class User(UserMixin, db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50))
    email = db.Column(db.String(255))
    password = db.Column(db.String(80))

try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)  
    
    def __repr__(self):
        return '<User %r>' % self.id
    

class Note(db.Model):
    __tablename__ = 'note'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e) 
    
    

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class SignupForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(),Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")
            
class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login") 
    
class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    note_body = StringField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Completion Date"})
    submit = SubmitField("Add Note")  

@app.route("/")
def home():
    status = "home"
    return render_template('index.html', status=status)


@app.route('/login', methods=["GET", "POST"])
def login():
    message=''
    status = "login"
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        print(username,password)

        user = User.query.filter_by(username=username).first()
        if not user:
            print("Wrong username or username not found.")
            flash("Wrong username or username not found.")
            return redirect(url_for('login'))
        else:
            if password == user.password:
                print("Successfully logged in.")
                login_user(user)
                flash("Successfully logged in.")
                return redirect(url_for('home'))
            else:
                print("Wrong password.")
                flash("Wrong password.")
                return redirect(url_for('login'))           
    else:
        return render_template('login.html',status=status)


@app.route("/signup", methods=["GET", "POST"])
def signup():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = User(username=username,email=email,password=password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('home'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('signup.html',status=status)

@app.route("/logout")
@login_required
def logout():
    print('logout user.')
    flash('logged out.', current_user.username)
    logout_user()
    return redirect(url_for('login')) 

@app.route('/newnote', methods=['GET','POST'])
@login_required
def newnote():
    form = NewNoteForm()
    if form.validate_on_submit():
        newnote = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user)
        db.session.add(newnote)
        db.session.commit()
        return redirect(url_for('mynotes'))
    return render_template('newnote.html', title='New Note', form=form)

@app.route('/mynotes', methods=['GET','POST'])
@login_required
def mynotes():
    notes = Note.query.all()
    return render_template('mynotes.html', notes=notes, title='My Notes')

@app.route('/delete_note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('mynotes'))

@app.route('/update/<int:note_id>', methods=['GET', 'POST'])
@login_required
def update_note(note_id):
    note = Note.query.get_or_404(note_id)

    if request.method == 'POST':
        note.title = request.form['Title']
        note.note_body = request.form['Note Body']
        db.session.commit()
        return redirect(url_for('mynotes'))
    return render_template('update.html', note=note, title='My Notes')


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == "__main__":
    app.run('localhost', 9033)

Database successfully created.
Exception occurred. Foreign key associated with column 'note.note_writer' could not find table 'user' with which to generate a foreign key to target column 'id'
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9033/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2021 16:00:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:00:24] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2021 16:00:25] "GET /newnote HTTP/1.1" 200 -
[2021-11-01 16:00:41,761] ERROR in app: Exception on /newnote [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_